In [1]:
import pandas as pd
import time
from selenium import webdriver
import csv
import requests
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
import threading
import math
import numpy as np
import os

In [6]:
if "key.csv" not in os.listdir():
    with open('./key.csv','w', newline='') as fd:
        writer = csv.writer(fd)
        writer.writerow(["caseno", "Key1","Key2"])
duration = 9000 # 要讓程式跑多久(秒)

In [2]:
duration = 300 # 要讓程式跑多久(秒)

In [2]:
# 分工
Maggie = (0, 80606)
job_range = Maggie

todo_df = pd.read_csv('todo.csv').iloc[job_range[0]:job_range[1], :]
key_df = pd.read_csv('key.csv')
# 從還沒完成的部分繼續爬
df = pd.merge(todo_df,key_df,on=['caseno'],how="outer",indicator=True)
df = df[df['_merge']=='left_only']
print("todo_df",todo_df)
print("key_df",key_df)
print("df",df)
df.to_csv("now.csv")

todo_df           caseno
0       89003693
1       89702625
2       89704172
3       89704174
4       89704175
...          ...
80601  109880763
80602  109880770
80603  109880791
80604  110880019
80605  110880067

[80606 rows x 1 columns]
key_df           caseno                                       Key1  \
0       89702625  XpJ13RyT4TGZFVnl2bVFvSWVSeDhldmVtT1F3QT09   
1       92054897  XpJ13RyT4QXJzM2JyY0NSV09SeDhldmVtT1F3QT09   
2       89704174  XpJ13RyT4NERudmVETDJaK2lSeDhldmVtT1F3QT09   
3       89704176  XpJ13RyT4QStWKzNuK2pRWFdSeDhldmVtT1F3QT09   
4       89704175  XpJ13RyT4NE9uVkdQNC8rZnlSeDhldmVtT1F3QT09   
...          ...                                        ...   
80592  108055700  XpJ13RyT4Y0NnNHBQUWRoQktSeDhldmVtT1F3QT09   
80593  108055701  XpJ13RyT4RVNmd0oyTE50Yk9SeDhldmVtT1F3QT09   
80594  108055717  XpJ13RyT4bFhhb0FJTG1JUnlSeDhldmVtT1F3QT09   
80595  108062740  XpJ13RyT4U0FJZG1lYkRuemFSeDhldmVtT1F3QT09   
80596  110880067  XpJ13RyT4eGs2NHRuNmt2R0NSeDhldmVtT1F3QT09   

In [4]:
class myThread(threading.Thread): #继承threading.Thread类
    def __init__(self,name,link_range):
        threading.Thread.__init__(self)
        self.name = name
        self.link_range = link_range
    def run(self):  #重写成员方法
        print ("Starting " + self.name)
        crawler(self.name, self.link_range)
        print ("Exiting " + self.name)

def crawler(threadName, link_range):
    CaseNos = now_df['caseno'].tolist()[link_range[0]: link_range[1]]
    print(threadName, link_range)
    fail_num = 0
    failcnt = 0
    
    option = webdriver.ChromeOptions()
    #option.add_argument('--headless')
    option.add_argument("--incogniton")   
    driver = webdriver.Chrome(chrome_options=option,executable_path='./chromedriver')
    
    #driver = webdriver.Chrome('./chromedriver')    
    start = time.time()
    print("start",start) 
    for i in range(0, len(CaseNos),10):
        print("{} Progress: {}/{}, Fail number: {}".format(threadName, i, len(CaseNos), fail_num))
        try:  
            driver.get("https://twtmsearch.tipo.gov.tw/OS0/OS0201.jsp?l6=zh_TW&isReadBulletinen_US=&isReadBulletinzh_TW=true")
            time.sleep(5+2*2)
            element = driver.find_element_by_id('applNo')
            #print("CaseNos",str(CaseNos[i]))
            element.send_keys(str(CaseNos[i]))
            for j in range(9):
                button = driver.find_element_by_xpath("//*[@id='form1']/fieldset/div[4]/div/table/tbody/tr/td[2]/img")
                button.click()
                time.sleep(0.005)
                temp = ('applNo'+"%s" % int(j+1))
                element = driver.find_element_by_id(temp)
                #print("CaseNos",str(CaseNos[i+j+1]))
                element.send_keys(str(CaseNos[i+j+1]))
            button = driver.find_element_by_id("querySubmit")
            button.click()
            time.sleep(2.5+2.5*2)

            alist_key1 = []
            alist_key2 = []
            blist = []
            for link in driver.find_elements_by_xpath("//*[@id='APPL_NO']/a"):
                #print("link",link.text)
                key1 = link.get_attribute('href')[29:70]
                key2 = link.get_attribute('href')[76:117]
                
                alist_key1.append(key1)
                alist_key2.append(key2)
                
                blist.append(link.text)
            failcnt = 0
        except:
            fail_num += 10
            failcnt += 1
            if failcnt == 10:
                break
            continue
        
        if(len(alist_key1) == len(alist_key2)):
            with open('./key.csv'.format(threadName),'a', newline='') as fd:
                writer = csv.writer(fd)
                for k in range(len(blist)):
                    writer.writerow([blist[k],alist_key1[k],alist_key2[k]])
        else:
            fail_num += 10
           
            failcnt += 1
            if failcnt == 10:
                break
            #continue

        end = time.time() 
        #print("end",end)   
        if(int(end-start) > duration):
            print("总体运行时间：",end-start)
            break

now_df = pd.read_csv('now.csv')
sep = len(now_df)//5
link_range=[(0,sep),(sep,sep*2),(sep*2,sep*3),(sep*3,sep*4),(sep*5,len(now_df))]  #建立5个线程异步
thread_list=[]

#创立新线程
for i in range(1,6): #Thread1-5
    thread=myThread('Thread-'+str(i),link_range[i-1])
    thread.start() #线程开始执行
    thread_list.append(thread)

#等待所有线程完成
for t in thread_list:
    t.join()

end=time.time()
print("Exiting the main")

Starting Thread-1
Thread-1 (0, 8)
Starting Thread-2
Thread-2 (8, 16)
Starting Thread-3
Thread-3 (16, 24)
Starting Thread-4
Thread-4 (24, 32)
Starting Thread-5
Thread-5 (40, 44)


C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: use options instead of chrome_options


start 1620027856.080936
Thread-3 Progress: 0/8, Fail number: 0
start 1620027856.8030736
Thread-1 Progress: 0/8, Fail number: 0
start 1620027856.837721
Thread-2 Progress: 0/8, Fail number: 0
start 1620027856.8607426
Thread-5 Progress: 0/4, Fail number: 0
start 1620027856.9340854
Thread-4 Progress: 0/8, Fail number: 0
Exiting Thread-5
Exiting Thread-4
Exiting Thread-1
Exiting Thread-2
Exiting Thread-3
Exiting the main


In [39]:
def crawlerz(CaseNos):   
    
    option = webdriver.ChromeOptions()
    #option.add_argument('--headless')
    option.add_argument("--incogniton")   
    driver = webdriver.Chrome(chrome_options=option,executable_path='./chromedriver')
    
    for i in range(0, len(CaseNos),10):
            
        try:  
            driver.get("https://twtmsearch.tipo.gov.tw/OS0/OS0201.jsp?l6=zh_TW&isReadBulletinen_US=&isReadBulletinzh_TW=true")
            time.sleep(5)
            element = driver.find_element_by_id('applNo')
            #print("CaseNos",str(CaseNos[i]))
            element.send_keys(str(CaseNos[i]))
            for j in range(9):
                button = driver.find_element_by_xpath("//*[@id='form1']/fieldset/div[4]/div/table/tbody/tr/td[2]/img")
                button.click()
                time.sleep(0.005)
                temp = ('applNo'+"%s" % int(j+1))
                element = driver.find_element_by_id(temp)
                #print("CaseNos",str(CaseNos[i+j+1]))
                element.send_keys(str(CaseNos[i+j+1]))
            button = driver.find_element_by_id("querySubmit")
            button.click()
            time.sleep(2.5)

            alist_key1 = []
            alist_key2 = []
            blist = []
            for link in driver.find_elements_by_xpath("//*[@id='APPL_NO']/a"):
                #print("link",link.text)
                key1 = link.get_attribute('href')[29:70]
                key2 = link.get_attribute('href')[76:117]

                alist_key1.append(key1)
                alist_key2.append(key2)

                blist.append(link.text)
                
            with open('left.csv','a' ,newline='') as fd:
                writer = csv.writer(fd)
                for k in range(len(blist)):
                    writer.writerow([blist[k],alist_key1[k],alist_key2[k]])
        except:
            print("Failed")

        print(alist_key1)
        print(alist_key2)
        print(blist)




In [44]:
crawlerz(x)

C:\Users\User\anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


Failed
[]
[]
[]
Failed
[]
[]
[]


In [43]:
with open('left.txt') as f:
    z = f.read()
    x = z.split()
    print(x)

['94041512', '94041514', '98013663', '100066613', '101002304', '101002305', '101051245', '102072091', '103009479', '94041512', '94041514', '98013663', '100066613', '101002304', '101002305', '101051245', '102072091', '103009479']


In [3]:
key_df.head()

,caseno,Key1,Key2,Unnamed: 3,Unnamed: 4
0,89702625,XpJ13RyT4TGZFVnl2bVFvSWVSeDhldmVtT1F3QT09,XpJ13RyT4U0FYbTBMcXhpV2RYREgxVVkraE9kdz09,NaN,NaN
1,92054897,XpJ13RyT4QXJzM2JyY0NSV09SeDhldmVtT1F3QT09,XpJ13RyT4Z0tNaGFGSEJSTEhTQUZwdzFoYnRxUT09,NaN,NaN
2,89704174,XpJ13RyT4NERudmVETDJaK2lSeDhldmVtT1F3QT09,XpJ13RyT4bm9UT212d1V3Y0JlTDZiZDM0RVM1QT09,NaN,NaN
3,89704176,XpJ13RyT4QStWKzNuK2pRWFdSeDhldmVtT1F3QT09,XpJ13RyT4bm9UT212d1V3Y0RHU0VlelRoYnF1UT09,NaN,NaN
4,89704175,XpJ13RyT4NE9uVkdQNC8rZnlSeDhldmVtT1F3QT09,XpJ13RyT4bm9UT212d1V3Y0JYREgxVVkraE9kdz09,NaN,NaN


In [14]:
a = key_df['caseno'].tolist()
x = [0] * 40
for i in a:
    x[i % 40] += 1

In [16]:
x

[2047,
 2041,
 2025,
 1999,
 1990,
 2038,
 2025,
 2027,
 1976,
 2067,
 1963,
 2032,
 2059,
 2000,
 1973,
 2009,
 1943,
 2036,
 2016,
 1945,
 1986,
 1998,
 1973,
 2073,
 1979,
 2091,
 1988,
 2076,
 2088,
 1975,
 1985,
 1965,
 2045,
 2033,
 2023,
 1937,
 2056,
 2075,
 2020,
 2020]